In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']


In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_mch4
(    
  mch4_id STRING,
  mch4_name STRING,
  mch4_desc STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
  
;""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW temp_d_mch4 AS
SELECT 
a._bic_zc_mchlv4 mch4_id,
a.txtsh mch4_name,
a.txtmd mch4_desc
FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY _bic_zc_mchlv4 ORDER BY proc_date DESC) rn 
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_zc_mchlv4
        WHERE end_date = '2400-01-01'
    ) a WHERE rn = 1""")

In [0]:
spark.sql(f''' 
CREATE OR REPLACE TEMP VIEW sap_zc_d_mch4 AS
    SELECT 
        a._bic_zc_mchlv4 mch4_id,  
        a.txtmd mch4_name,
        a.txtmd mch4_desc
FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY _bic_zc_mchlv4 ORDER BY proc_date DESC) rn 
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_zc_mchlv4
        WHERE end_date = '2400-01-01'
    ) a WHERE rn = 1 

          ''')

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_mch4 AS main_tbl
USING temp_d_mch4 AS temp_tbl
ON main_tbl.mch4_id = temp_tbl.mch4_id 

WHEN MATCHED THEN 
    UPDATE SET 
        main_tbl.mch4_name = temp_tbl.mch4_name,
        main_tbl.mch4_desc = temp_tbl.mch4_desc

WHEN NOT MATCHED THEN 
    INSERT (mch4_id, mch4_name, mch4_desc)
    VALUES (temp_tbl.mch4_id, temp_tbl.mch4_name, temp_tbl.mch4_desc)
    """)

In [0]:
spark.sql("DROP VIEW IF EXISTS temp_d_mch4")
spark.sql("DROP VIEW IF EXISTS sap_zc_d_mch4")